In [2]:
import os 
# os.chdir("C:\\Users\\Joseph\\Active Github Repositories\\pneumothorax-seg-cnn")
os.chdir("C:\\Users\\Joseph\\Active Github Repositories\\efficientnet-pneumothorax-seg")
os.getcwd()

'C:\\Users\\Joseph\\Active Github Repositories\\efficientnet-pneumothorax-seg'

# RSNA Grayscale models

In [3]:
from efficientnet_seg.models.grayscale.densenet import DenseNet121, DenseNet169, DenseNet201 
from efficientnet_seg.models.grayscale.xception import Xception 
from efficientnet_seg.models.grayscale.inception_resnet_v2 import InceptionResNetV2 

from tensorflow.keras.layers import Dropout, Flatten, Dense, Input, Concatenate, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras import backend as K 
from tensorflow.keras import optimizers, layers, utils

def get_model(base_model, 
              layer, 
              lr=1e-3, 
              input_shape=(224,224,1), 
              classes=1,
              activation="sigmoid",
              dropout=None, 
              pooling="avg", 
              weights=None,
              pretrained="imagenet"): 
    base = base_model(input_shape=input_shape,
                      include_top=False,
                      weights=pretrained, 
                      ) 
    if pooling == "avg": 
        x = GlobalAveragePooling2D()(base.output) 
    elif pooling == "max": 
        x = GlobalMaxPooling2D()(base.output) 
    elif pooling is None: 
        x = Flatten()(base.output) 
    if dropout is not None: 
        x = Dropout(dropout)(x) 
    x = Dense(classes, activation=activation)(x) 
    model = Model(inputs=base.input, outputs=x) 
    if weights is not None: 
        model.load_weights(weights) 
    for l in model.layers[:layer]:
        l.trainable = False 
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], 
                  optimizer=optimizers.Adam(lr)) 
    return model

In [9]:
import os

# from tensorflow.keras.applications import DenseNet169
weights_dir = "C:\\Users\\Joseph\\Pretrained Weights\\RSNA Pneumonia\\models\\pretrained"
# weights_path = os.path.join(weights_dir, "DenseNet169_NIH15_Px448.h5") # Works; 12M
# weights_path = os.path.join(weights_dir, "InceptionResNetV2_NIH15_Px256.h5") # Works; 55M
# weights_path = os.path.join(weights_dir, "Xception_NIH15_Px320.h5") # Works; 20.8 M
input_size = 320
model = get_model(Xception, 0, 5e-5, dropout=None, input_shape=(input_size, input_size, 1),
                  pretrained=weights_path)
# model.load_weights(weights_path)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 320, 320, 1)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 159, 159, 32) 288         input_4[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 159, 159, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 159, 159, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

# WRN

In [4]:
import pneumothorax_seg
from pneumothorax_seg.models.cascaded_model.wrn.models import WideResidualNetwork

import tensorflow.keras.backend as K
import tensorflow.keras as keras

K.set_image_data_format("channels_last")
dformat = K.image_data_format()
print("Data format: {0}".format(dformat))

# 28-8
# 23,372,385
# 23,372,385

n_classes = 1
# build model
s = 1024
shape = (s, s, 1) if K.image_data_format() == "channels_last" else (1, s, s)
model = WideResidualNetwork(depth=28, width=8, dropout_rate=0.0,
                            include_top=True, weights=None,
                            input_shape=shape,
                            classes=1, activation='sigmoid')
# train
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Data format: channels_last
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1024, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d_56 (Conv2D)              (None, 1024, 1024, 1 160         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1_50 (Batc (None, 1024, 1024, 1 64          conv2d_56[0][0]                  
__________________________________________________________________________________________________
activation_50 (Activation)      (None, 1024, 1024, 1 0           batch_normalization_v1_50[0][0]  
__________________________________________________________________________________

# EfficientNet

In [4]:
import pneumothorax_seg
from pneumothorax_seg.models.cascaded_model.efficientnet.models import EfficientNetB3

import tensorflow.keras.backend as K
import tensorflow.keras as keras

K.set_image_data_format("channels_last")
dformat = K.image_data_format()
print("Data format: {0}".format(dformat))

# B3
# Total params: 10,784,345
# Trainable params: 10,697,049
# Non-trainable params: 87,296
n_classes = 1
# build model
s = 1024
shape = (s, s, 1) if K.image_data_format() == "channels_last" else (1, s, s)
base_model = EfficientNetB3(input_shape=shape, weights=None, include_top=False, data_format=dformat)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='sigmoid')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])
# train
model.compile(optimizer='SGD', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Data format: channels_last
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 40) 360         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 512, 512, 40) 160         conv2d[0][0]                     
_________________________

# Training

In [5]:
from pneumothorax_seg.io.generators import Slim2DGenerator, ClassificationGenerator
from pneumothorax_seg.io.io_utils import split_ids, remove_broken_dirty_fnames

import pandas as pd
from glob import glob
import json
import os

base_dir = "C:\\Users\\Joseph\\Datasets\\SIIM Pneumothorax Segmentation 2019\\"
rle_csv_path = os.path.join(base_dir, "train-rle.csv")
train_fpaths = sorted(glob(base_dir+'dicom-images-train\\*\\*\\*.dcm'))
test_fpaths = sorted(glob(base_dir+'dicom-images-test\\*\\*\\*.dcm'))

df = pd.read_csv(rle_csv_path, header=None, index_col=0) 
df2 = pd.read_csv(rle_csv_path)
df_ids = df2["ImageId"].tolist()

In [6]:
try:
    fname = "fold1_802000.json"
    with open(fname, "r") as fp:
        id_dict = json.load(fp)
except FileNotFoundError:
    fixed_fpaths = remove_broken_dirty_fnames(train_fpaths, df_ids, df)
    print(len(fixed_fpaths), len(df_ids))
    id_dict = split_ids(fixed_fpaths, splits=[0.8, 0.2, 0])
len(id_dict['train']), len(id_dict['val'])

10051 11582


(8041, 2010)

In [7]:
from batchgenerators.transforms.abstract_transforms import AbstractTransform
import numpy as np

class ConvertToChannelLastTransform(AbstractTransform):
    def __init__(self, input_keys):
        """
        converts all keys listed in input_keys from (b, c, x, y(, z)) to (b, x, y(, z), c).
        """
        self.input_keys = input_keys

    def __call__(self, **data_dict):
        for k in self.input_keys:
            data = data_dict.get(k)
            if data is None:
                print("WARNING in ConvertToChannelLastTransform: data_dict has no key named", k)
            else:
                if len(data.shape) == 4:
                    new_ordering = (0, 2, 3, 1)
                elif len(data.shape) == 5:
                    new_ordering = (0, 2, 3, 4, 1)
                else:
                    raise RuntimeError("unsupported dimensionality for ConvertToChannelLastTransform:",
                                       len(data.shape),
                                       ". Only 2d (b, c, x, y) and 3d (b, c, x, y, z) are supported for now.")
                assert isinstance(data, np.ndarray), "data_dict[k] must be a numpy array"
                data = data.transpose(new_ordering)
                data_dict[k] = data
        return data_dict

In [8]:
channels_last = ConvertToChannelLastTransform(["data"])
gen = ClassificationGenerator(id_dict["train"], rle_csv_path, batch_size=2,
                             transform=channels_last, steps_per_epoch=None, shuffle=True)

Adjusting the indexes since the total number of required samples (steps_per_epoch * batch_size) is greater than the number of provided images.
Done!


In [9]:
%time x, y = gen[0]
x.shape, y.shape

Wall time: 814 ms


((2, 1024, 1024, 1), (2, 1))

In [10]:
# efficientnetb3
model.fit(x[0:1], y[0:1], batch_size=1, epochs=1)

Instructions for updating:
Use tf.cast instead.
1/1 [==============================] - 109s 109s/sample - loss: 0.5109 - acc: 1.0000


In [ ]:
# wrn
model.fit(x[0:1], y[0:1], batch_size=1, epochs=1)

Instructions for updating:
Use tf.cast instead.
